<a href="https://colab.research.google.com/github/lonelu/Rvrsprot/blob/main/Colab_Rvrsprot_master_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rvrsprot_master_loop_colab
DeGrado Lab, UCSF.

The program is used for loop selections from Master result. 

In [ ]:
#@title Upload pdb
import os
from google.colab import files
import re
import datetime
import shutil
#@markdown Define job
jobname = 'None' #@param {type:"string"}

#@markdown Upload pdb prompt
uploaded = files.upload()
for fn in uploaded.keys():
  # remove any preceding '.'s in filename
  count = fn.count(".") - 1
  newfn = fn.replace('.', '-', count)
  newfn = fn.replace('_', '-')
pdbname = newfn.split('.')[0]
shutil.copyfile(fn, pdbname + '.pdb')



Saving 1f4m_CCCP_ALA.pdb to 1f4m_CCCP_ALA.pdb


'1f4m-CCCP-ALA.pdb'

In [ ]:
#@title Set parameters
import os
from google.colab import files
import re
import datetime

if jobname == 'None':
  jobname = pdbname

time_tag = datetime.datetime.now().strftime('%Y%m%d-%H%M%S') 

workdir = '/content/' + jobname + '_' + time_tag 
if not os.path.exists(workdir):
  os.mkdir(workdir)

#@markdown Set parameter

loop_range_min = 11 #@param {type:"integer"}
loop_range_max = 13 #@param {type:"integer"}

loop_target_list = 'master_origin' #@param {type:"string"}
rmsdCut = 1.5 #@param {type:"number"} 

master_query_loop_top = 500 #@param {type:"integer"}
cluster_rmsd = 1.0 #@param {type:"number"}
rm_duplicate = True #@param {type:"boolean"} 
#@markdown - (if the database have duplicated pdbs.)

loop_topo_sels_front = 'A-36-42-B-17-23, B-36-42-C-17-23' #@param {type:"string"}
loop_topo_sels_back = 'None' #@param {type:"string"}

cluster_count_cut = 10 #@param {type:"number"}
select_min_rmsd_pdb= True #@param {type:"boolean"} 
#@markdown - (output will copy the result pdb with min rmsd.)

class Para():
  workdir = workdir
  target_file = pdbname

  loop_topo_sels = {}

  if loop_topo_sels_front != 'None' and loop_topo_sels_back != '':
    xx = re.sub(r'\s+', '', loop_topo_sels_front)
    xx = xx.split(',')
    front = {}
    for x in xx:
      _xs = x.split('-')
      key = _xs[0] + '-' + _xs[3]
      if key in front.keys():
        front[key].append((_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5])))
      else:
        front[key] = [(_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5]))]
    loop_topo_sels['front'] = front

  if loop_topo_sels_back != 'None' and loop_topo_sels_back != '':
    xx = re.sub(r'\s+', '', loop_topo_sels_back)
    xx = xx.split(',')
    back = {}
    for x in xx:
      _xs = x.split('-')
      key = _xs[0] + '-' + _xs[3]
      if key in back.keys():
        back[key].append((_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5])))
      else:
        back[key] = [(_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5]))]
    loop_topo_sels['back'] = back   

  ### Master search para
  loop_range = [loop_range_min, loop_range_max]
  if loop_target_list == 'master_origin':
    loop_target_list = '/content/master_db/list'

  rmsdCut = rmsdCut
  master_query_loop_top = master_query_loop_top
  cluster_rmsd = cluster_rmsd
  rm_duplicate = rm_duplicate

  ### Output para
  cluster_count_cut = cluster_count_cut
  select_min_rmsd_pdb= select_min_rmsd_pdb



In [ ]:
#@title Install Rvrsprot.

%%bash -s

set -e

cd /content

#rm -rf RVRSPROT_READY METALPROT_READY

if [ ! -f RVRSPROT_READY ]; then
  echo "Cloning Rvrsprot..."
  rm -rf Rvrsprot
  git clone https://github.com/lonelu/Rvrsprot.git
  cd /content/Rvrsprot
  pip install -e .
  touch RVRSPROT_READY
fi

cd /content/

if [ ! -f METALPROT_READY ]; then
  echo "Cloning Metalprot..."
  rm -rf Metalprot
  git clone https://github.com/lonelu/Metalprot.git

  cd /content/Metalprot
  pip install -e .
  touch METALPROT_READY
fi

chmod 777 /content/Rvrsprot/rvrsprot/external/createPDS
chmod 777 /content/Rvrsprot/rvrsprot/external/parsePDS
chmod 777 /content/Rvrsprot/rvrsprot/external/master


#download database and change list filepath.
pip install logomaker
pip install fpdf


Cloning Rvrsprot...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/Rvrsprot
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached biopython-1.76-cp37-cp37m-manylinux1_x86_64.whl (2.3 MB)
  Created wheel for prody: filename=ProDy-2.2.0-cp37-cp37m-linux_x86_64.whl size=10816258 sha256=2b4df01f3b9a23f93bf19d3859abac46398eb640c3a9bef5f39da11c862d27df
  Stored in directory: /root/.cache/pip/wheels/fc/59/47/97758bdbc4d5edf0aad1f7ccfeac5c687f6c2694d5efdb7571
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=151a225fdbd1bc30ff7524c61c8fc68191eb2b35923c757b394b842b80f9f8c3
  Stored in directory: /root/.cache/pip/

Cloning into 'Rvrsprot'...
Cloning into 'Metalprot'...


In [ ]:
#@title Download database
%%bash -s

set -e

if [ ! -f DB_READY ]; then
  rm -rf /content/master_db.zip
  curl -L https://ucsf.box.com/shared/static/3wabh6hfii8fl3bui9txpox9l9rphygv.zip --output /content/master_db.zip 
  ls -l /content/master_db.zip
  unzip -q /content/master_db.zip
  touch DB_READY
fi

if [ ! -f DB_READY2 ]; then
  #unzip -q /content/master_db.zip
  ls -l /content
  sed -i 's/\/wynton\/home\/degradolab\/lonelu\/DesignData\/Database/\/content/g' /content/master_db/list
  head /content/master_db/list
  touch DB_READY2
fi



total 1357620
drwxr-xr-x   2 root root       4096 Oct 26 04:14 1f4m-CCCP-ALA_20221026-041450
-rw-r--r--   1 root root      56408 Oct 26 04:14 1f4m_CCCP_ALA.pdb
-rw-r--r--   1 root root      56408 Oct 26 04:14 1f4m-CCCP-ALA.pdb
-rw-r--r--   1 root root          0 Oct 26 04:17 DB_READY
drwxr-xr-x 941 root root      20480 Oct 26 04:17 master_db
-rw-r--r--   1 root root 1390044745 Oct 26 04:17 master_db.zip
drwxr-xr-x   9 root root       4096 Oct 26 04:15 Metalprot
drwxr-xr-x   9 root root       4096 Oct 26 04:15 Rvrsprot
drwxr-xr-x   1 root root       4096 Oct 24 13:33 sample_data
/content/master_db/55/155c_A.pds
/content/master_db/6v/16vp_A.pds
/content/master_db/91/1914_A.pds
/content/master_db/a0/1a02_F.pds
/content/master_db/a0/1a02_N.pds
/content/master_db/a0/1a0a_A.pds
/content/master_db/a0/1a0b_A.pds
/content/master_db/a0/1a0e_A.pds
/content/master_db/a0/1a0g_A.pds
/content/master_db/a0/1a0i_A.pds


In [ ]:
#@title Run master loop.

import os
import sys
sys.path.append(r'/content/Metalprot')
sys.path.append(r'/content/Rvrsprot')
from rvrsprot.loop import loop_ss
import datetime
import subprocess


para = Para()
def run():
    #para = Para()
    loop_ss.run_loop_ss(para.workdir + '/', '/content/' + para.target_file + '.pdb', para.loop_topo_sels, para)
    return

run()  
!cd /content/
!zip -FSr $para.workdir".zip" $para.workdir"/"

files.download(para.workdir + '.zip')

@> 4 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:4 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 5 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:5 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 10 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:10 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 696 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:696 atoms and 1 coordinate set(s) were parsed in 0.01s.


Querying MASTER for loops of length 3 to 13.


@> 206 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:206 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 208 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:208 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 169 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:169 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 190 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:190 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 205 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:205 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 188 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:188 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 178 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:178 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 192 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:192 atoms and 1 coordinate set(s

Sorting loop PDBs by loop length./content/1f4m-CCCP-ALA_20221026-041450/front/A-36-42-B-17-23/


@> 188 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:188 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 216 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:216 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 195 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:195 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 193 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:193 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 327 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:327 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 204 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:204 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 177 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:177 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 215 atoms and 1 coordinate set(s) were parsed in 0.01s.
DEBUG:.prody:215 atoms and 1 coordinate set(s

...

fail _plot_log


@> 218 atoms and 1 coordinate set(s) were parsed in 0.00s.
DEBUG:.prody:218 atoms and 1 coordinate set(s) were parsed in 0.00s.


fail _plot_log
  adding: content/1f4m-CCCP-ALA_20221026-041450/ (stored 0%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/ (stored 0%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/ (stored 0%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/B-36-42-C-17-23.pds (deflated 67%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/match.txt (deflated 72%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/stdout (deflated 88%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/9/ (stored 0%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/9/match184.pdb (deflated 72%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/9/match383.pdb (deflated 73%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/9/match375.pdb (deflated 73%)
  adding: content/1f4m-CCCP-ALA_20221026-041450/front/B-36-42-C-17-23/9/match180.pdb (deflated 72

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The program is for DeGrado lab internal usage now (10/25/2022).

Please contact lonelur@gmail.com if you find something wrong or if you have special suggestions to help other researchers.